In [18]:
# INSTALL ANY PYTHON PACKAGE YOU NEED HERE!!
# restart the kernel after installing new pkgs

# %pip install geopandas 
# %pip install matplotlib
# %pip install pyspark_dist_explore
# %pip install plotly
# %pip install chart-studio
# %pip install pysnmp

SparkConf created
Started SparkSession
Spark version 3.5.0


In [ ]:
%config Completer.use_jedi = False
%run spark-instance.ipynb

In [19]:
# system imports
import os
import time 
from datetime import datetime

# installed pkg imports
import numpy as np
import pandas as pd
import ipaddress

pd.set_option('display.max_rows', 300)

# spark imports
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
from pyspark.sql.functions import * 
from pyspark.sql.functions import monotonically_increasing_id

# Other imports 
from pyspark.sql.functions import col, udf 
from pyspark.sql.types import BooleanType
import matplotlib.pyplot as plt
from pyspark_dist_explore import hist
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'

# local python files imports

In [20]:
# you have access to the following dates:
#year=2023/month=12/day=19 - 5
#year=2024/month=01/day=16 - 6
#year=2024/month=02/day=20 - 7
#year=2024/month=03/day=12 - 8

#Change the value of date between 1-8. 

ts = datetime(2023, 12, 19)

date = 8

match date:
    case 5:
        ts = datetime(2023, 12, 19)
    case 6:
        ts = datetime(2024, 1, 16)
    case 7:
        ts = datetime(2024, 2, 20)
    case 8:
        ts = datetime(2024, 3, 12)
    
CENSYS_BASE_PATH_FMT = "../dataset/censys/dataset=universal-internet-dataset-v2-ipv4/format=parquet/"
CENSYS_PATH_FMT = os.path.join(CENSYS_BASE_PATH_FMT, "year={year}/month={month:02d}/day={day:02d}")

In [21]:
censys_path = CENSYS_PATH_FMT.format(year=ts.year, month=ts.month, day=ts.day)
censys_df = spark.read.option("basePath", CENSYS_BASE_PATH_FMT).parquet(censys_path)

In [23]:
# Useful methods applied during research.

def is_list_all_none(lst):
    if isinstance(lst, list):
        return all(x is None for x in lst)
    return False

def extract_value(lst):
    if isinstance(lst, list):
        for item in lst:
            if item is not None:
                return item
    return None

def extract_list_with_strings(lst_of_lsts):
    if isinstance(lst_of_lsts, list):
        for sublist in lst_of_lsts:
            if isinstance(sublist, list) and any(isinstance(item, str) for item in sublist):
                return sublist
    return None
    
def count_versions(sublists, version):
    if sublists is not None:
        count = 0
        for sublist in sublists:
            if sublist is not None:
                count += sublist.count(version)
        return count
    return 0


def categorize_list(lst):
    if set(lst) == {'v1', 'v2c', 'v3'}:
        return 'v1, v2, v3'
    elif set(lst) == {'v1'}:
        return 'Just v1'
    elif set(lst) == {'v2c'}:
        return 'Just v2c'
    elif set(lst) == {'v3'}:
        return 'Just v3'
    elif set(lst) == {'v1', 'v2c'}:
        return 'Just v1 and v2c'
    elif set(lst) == {'v1', 'v3'}:
        return 'Just v1 and v3'
    elif set(lst) == {'v2c', 'v3'}:
        return 'Just v2c and v3'
    return 'other'

def remove_nones(lst):
    if isinstance(lst, list):
        return [item for item in lst if item is not None]
    return lst